# Персептрон
## Распознавание символов S и 4

_Импорт библиотек_

In [134]:
import numpy as np
import scipy.special
from random import randint
import random
from PIL import Image
import math

CONST_IMG_SIZE = 100

_Функция преобразования изображения в массив из 0 и 1_

In [135]:
def imageToArray(filename):
    openedImage = Image.open(filename)
    grayColorImage = openedImage.convert('L')
    bwColorImage = np.asarray(grayColorImage).copy()

    for i in range(int(len(bwColorImage)/2)):
        for j in range(int(len(bwColorImage)/2)):
            bwColorImage[i][j] = 1 if bwColorImage[i][j] == 0 else 0
    
    return bwColorImage

_Функция для масштабирования входного изображения_

In [136]:
def scaleImage(arrayFromImage, n):
    tmp = [[0 for x in range(n)] for y in range(n)] 
            
    xmin = n
    xmax = -1
    ymin = n 
    ymax = -1
    
    for i in range(n):
        for j in range(n):
            if arrayFromImage[i][j] == 1:
                if j < xmin: xmin = j
                if j > xmax: xmax = j
                if i < ymin: ymin = i
                if i > ymax: ymax = i
                    
            tmp[i][j] = arrayFromImage[ymin + int((i / n) * (ymax - ymin + 1))][xmin + int((j / n) * (xmax - xmin + 1))]
            
            arrayFromImage[i][j] = tmp[i][j]
         
    return arrayFromImage

_Функция обработки входного изображения_

In [137]:
def imageProcessing(filename):
    arrayFromImage = imageToArray(filename)
    scaledImage = scaleImage(arrayFromImage, CONST_IMG_SIZE)
    
    return scaledImage.ravel()

_Создание обучающей выборки_

In [138]:
listToLearn_S = []
listToLearn_4 = []

listToTest = []

# Заполняем данные для обучения
for num in range(1, 23):
    # массивы картинок с символом S 
    filename_S = "~/Learn - Symbol S/" + str(num) + '.png'
    listToLearn_S.append(imageProcessing(filename_S))
    
    # массивы картинок с символом 4 
    filename_4 = "~/Learn - Symbol 4/" + str(num) + '.png'
    listToLearn_4.append(imageProcessing(filename_4))

_Заполнение массива весов случайными значениями из диапазона [-0.1, 0.1]_

In [163]:
imageSize = CONST_IMG_SIZE * CONST_IMG_SIZE
weights = [0 for x in range(imageSize)]

for i in range(imageSize):
    weights[i] = random.uniform(-0.1, 0.1)


_Функция тренировки сети_

In [140]:
# Коэффициент скорости обучения
speed = 0.7
# Порог функции активации
bias = 5

def trainNet(symbol, rightAnswer):
    # Рассчитываем взвешенную сумму
    net = 0
    for i in range(imageSize):
        net += symbol[i] * weights[i]
        
    # Превышен ли порог? (Да - сеть думает, что это S. Нет - сеть думает, что это 4)
    perceptonResult = 1 if net >= bias else 0
    
    # Сопоставляем ответ сети с корректным ответом
    checkPerceptonResult = perceptonResult == rightAnswer

    print("perceptonResult ", perceptonResult, " rightAnswer ", rightAnswer, " checkPerceptonResult ",checkPerceptonResult)
    
    # Если сеть ошиблась и не распознала символ
    if checkPerceptonResult == False:
        weightCorrectionValue = 1 if perceptonResult == 0 else -1
        
        for i in range(imageSize):
            weights[i] = weights[i] + speed * weightCorrectionValue * symbol[i]

_Функция распознования символов_

In [141]:
def startDetection(symbol):
    # Рассчитываем взвешенную сумму
    net = 0
    for i in range(imageSize):
        net += symbol[i] * weights[i]
        
    # Превышен ли порог? (Да - сеть думает, что это S. Нет - сеть думает, что это 4)
    perceptonResult = 1 if net >= bias else 0
    
    return perceptonResult

_Тренировка сети_

In [164]:
j = 0 
learningRange = 44
arrayOfRandomIndeces = []

for i in range(int(learningRange / 2)):
    arrayOfRandomIndeces.append(0)
    arrayOfRandomIndeces.append(1)
    
random.shuffle(arrayOfRandomIndeces)

for i in range(learningRange):
    randomIndex = arrayOfRandomIndeces[i]
    listsForRandom = [listToLearn_4[j], listToLearn_S[j]]
    trainNet(listsForRandom[randomIndex], randomIndex)
 
    if (j + 1) == 21:
        j = 0
    else:
        j += 1
        
print("Тренировка сети завершена!")

perceptonResult  1  rightAnswer  0  checkPerceptonResult  False
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  0  rightAnswer  1  checkPerceptonResult  False
perceptonResult  0  rightAnswer  1  checkPerceptonResult  False
perceptonResult  1  rightAnswer  0  checkPerceptonResult  False
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  0  rightAnswer  1  checkPerceptonResult  False
perceptonResult  1  rightAnswer  0  checkPerceptonResult  False
perceptonResult  1  rightAnswer  1  checkPerceptonResult  True
perceptonResult  0  rightAnswer  0  checkPerceptonResult  True
perceptonResult  1  rightAnswer  0  checkPerceptonResult  False
perceptonResult  0  rightAnswer  1  checkPercept

_Прогон по обучающей выборке_

In [165]:
for num in range(5):
    result = startDetection(listToLearn_S[num])
    print("S | ответ сети: ", "s" if result == 1 else "4" )

    result = startDetection(listToLearn_4[num])
    print("4 | ответ сети: ", "s" if result == 1 else "4" )
    print('\n')

S | ответ сети:  s
4 | ответ сети:  4


S | ответ сети:  s
4 | ответ сети:  4


S | ответ сети:  s
4 | ответ сети:  4


S | ответ сети:  s
4 | ответ сети:  4


S | ответ сети:  s
4 | ответ сети:  4




_Прогон по тестовой выборке_

In [171]:
filename_S = "~/Test - Symbol S/2.png"

result = startDetection(imageProcessing(filename_S))
print("S | ответ сети: ", "s" if result == 1 else "4" )


S | ответ сети:  s


In [174]:
filename_4 = "~/Test - Symbol 4/3.png"

result = startDetection(imageProcessing(filename_4))
print("4 | ответ сети: ", "s" if result == 1 else "4" )

4 | ответ сети:  4
